In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import keras
import tensorflow as tf
# from tensorflow.keras.applications import VGG16
from keras.utils.vis_utils import plot_model
import os, shutil
from keras import models
from keras import layers
from tensorflow.keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras import models
from keras.preprocessing import image
import numpy as np
import sklearn.metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import itertools
import seaborn

print(keras.__version__)
print(tf.__version__)

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os

import cv2
import matplotlib.pyplot as plt

Download the dataset

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1KO0ryOH6j4pFYTJpSyGxL1iH1f84dNU7&export=download

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1j9nLFoAA5xxC5DplQd-mgbysVsMcJEcz&export=download

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1MWLGbv82_aEZo3h84pQAMRHkvCvsWSkA&export=download

In [ ]:
!unzip /content/1.zip

**Rename the zip files**

In [ ]:
!unzip /content/0.zip

In [ ]:
!unzip /content/2_3_4.zip

In [ ]:
!mkdir weather_detection
%cd /content/weather_detection
!mkdir train
!mkdir test
!mkdir validation
%cd /content/weather_detection/test
!mkdir Snowy
!mkdir Cloudy
!mkdir Sunny
!mkdir Rainy
!mkdir Foggy
%cd /content/weather_detection/train
!mkdir Snowy
!mkdir Cloudy
!mkdir Sunny
!mkdir Rainy
!mkdir Foggy
%cd /content/weather_detection/validation
!mkdir Snowy
!mkdir Cloudy
!mkdir Sunny
!mkdir Rainy
!mkdir Foggy
%cd /content

In [ ]:
print(len(os.listdir('/content/0')))
print(len(os.listdir('/content/1')))
print(len(os.listdir('/content/2')))
print(len(os.listdir('/content/3')))
print(len(os.listdir('/content/4')))

**Splitting Dataset into Train Test Validate sets**

In [ ]:
x_train,x_test,x_val=[],[],[]
print('Train')
l={'/content/0':1000,'/content/1':1000,'/content/2':800,'/content/3':700,'/content/4':300}
k=['/content/weather_detection/train/Cloudy','/content/weather_detection/train/Sunny','/content/weather_detection/train/Rainy','/content/weather_detection/train/Snowy','/content/weather_detection/train/Foggy']
for src,dst in zip(l.keys(),k):
  i=1
  for img in os.listdir(src):
    shutil.move(src+f'/{img}',dst+f'/{img}')
    img_arr=cv2.imread(dst+f'/{img}')
    img_arr=cv2.resize(img_arr,(100,100))
    x_train.append(img_arr)
    if i==l[src]:
      break
    i+=1
for i in k:
  t=i.split('/')[-1]
  print(t,':',len(os.listdir(i)))


print('Validation')
l={'/content/0':500,'/content/1':500,'/content/2':254,'/content/3':152,'/content/4':121}
k=['/content/weather_detection/validation/Cloudy','/content/weather_detection/validation/Sunny','/content/weather_detection/validation/Rainy','/content/weather_detection/validation/Snowy','/content/weather_detection/validation/Foggy']
for src,dst in zip(l.keys(),k):
  i=1
  for img in os.listdir(src):
    shutil.move(src+f'/{img}',dst+f'/{img}')
    
    img_arr=cv2.imread(dst+f'/{img}')
    img_arr=cv2.resize(img_arr,(100,100))
    x_val.append(img_arr)

    if i==l[src]:
      break
    i+=1
for i in k:
  t=i.split('/')[-1]
  print(t,':',len(os.listdir(i)))


print('Test')
l={'/content/0':800,'/content/1':800,'/content/2':400,'/content/3':400,'/content/4':200}
k=['/content/weather_detection/test/Cloudy','/content/weather_detection/test/Sunny','/content/weather_detection/test/Rainy','/content/weather_detection/test/Snowy','/content/weather_detection/test/Foggy']
for src,dst in zip(l.keys(),k):
  i=1
  for img in os.listdir(src):
    shutil.move(src+f'/{img}',dst+f'/{img}')
    

    img_arr=cv2.imread(dst+f'/{img}')
    img_arr=cv2.resize(img_arr,(100,100))
    x_test.append(img_arr)

    if i==l[src]:
      break
    i+=1
for i in k:
  t=i.split('/')[-1]
  print(t,':',len(os.listdir(i)))

Converting Images Array into Numpy Arrays

In [ ]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

########################################

train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

path to test,train and validation directories

In [ ]:
train_path='/content/weather_detection/train/'
test_path='/content/weather_detection/test'
val_path='/content/weather_detection/validation/'

Defining Data Generators

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)



training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (100, 100),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (100, 100),
                                            batch_size = 32,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (100, 100),
                                            batch_size = 32,
                                            class_mode = 'sparse')

In [ ]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [ ]:
training_set.class_indices

In [ ]:
train_y.shape,test_y.shape,val_y.shape

Loading Model

In [ ]:
model = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False) # Shape of our images

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable = False

For Binary Classification

In [ ]:
# # Flatten the output layer to 1 dimension
# x = layers.Flatten()(model.output)

# # Add a fully connected layer with 512 hidden units and ReLU activation
# x = layers.Dense(512, activation='relu')(x)

# # Add a dropout rate of 0.5
# x = layers.Dropout(0.5)(x)

# # Add a final sigmoid layer with 1 node for classification output
# x = layers.Dense(1, activation='sigmoid')(x)
# model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

For Multi-class Classificatiion

In [ ]:
x = Flatten()(model.output)
prediction = Dense(5, activation='softmax')(x)

model = Model(inputs=model.input, outputs=prediction)

model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

Defining Early stopping

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

Training the model using transfer learning

In [ ]:
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=100,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)

Analysing and Visualizing Training and validation results

In [ ]:
plt.plot(history.history['accuracy'], label='train acc')

plt.plot(history.history['val_accuracy'], label='val acc')

plt.legend()

plt.savefig('vgg-acc-rps-1.png')

plt.show()

In [ ]:
# loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('vgg-loss-rps-1.png')
plt.show()

Save the model

In [ ]:
model.save('/content/drive/MyDrive/weather_detection/trained_model/vgg16_wheather_detection.h5')

Evalute the model on the test set

In [ ]:
model.evaluate(test_x,test_y,batch_size=32)

Analysing the Testing results

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np
y_pred=model.predict(test_x)
y_pred=np.argmax(y_pred,axis=1)
#get classification report
print(classification_report(y_pred,test_y))
#get confusion matrix
print(confusion_matrix(y_pred,test_y))

**Analysing Failure test cases from saved models**

**Dataset 1**

In [ ]:
!unzip /content/drive/MyDrive/weather_detection/Weather_detection_dataset.zip

**Dataset 2**

In [ ]:
!unzip /content/drive/MyDrive/weather_detection/dataset2new.zip

**Loading Saved Model**

In [ ]:
model = models.load_model('/content/drive/MyDrive/weather_detection/trained_model/2_DenseNet201_transf2.h5')

In [ ]:
model.summary()

Model evalution

In [ ]:
model.evaluate(test_x,test_y,batch_size=32)

Seprating the Failure Test Cases

In [ ]:
%matplotlib inline
from keras.preprocessing import image

Testing one image at a time

In [ ]:
img_path = '/content/alien_test/foggy_8.jpg'

img = image.load_img(img_path,target_size=(224, 224, 3))
# Convert it to a Numpy array with shape (150, 150, 3)
test_x = image.img_to_array(img)
# Reshape it to (1, 150, 150, 3)
test_x = test_x.reshape((1,) + test_x.shape)
test_x = test_x.astype('float32') / 255

predict = model.predict(np.array(test_x))
output = {0:'Cloudy',1:'Foggy',2:'Rainy',3:'Snowy',4:'Sunny'}
plt.title(output[np.argmax(predict)])  
plt.imshow(img)

Testing all images and seprate the failure test cases

In [ ]:
src='/content/Rainy'
dst='/content/drive/MyDrive/weather_detection/Failed_DenseNet_dataset2/Rainy'
k=0
for name in os.listdir(src):
  img_path = src+'/'+f'{name}'
  img = image.load_img(img_path,target_size=(224, 224, 3))
  # Convert it to a Numpy array with shape (150, 150, 3)
  test_x = image.img_to_array(img)
  # Reshape it to (1, 150, 150, 3)
  test_x = test_x.reshape((1,) + test_x.shape)
  test_x = test_x.astype('float32') / 255

  predict = model.predict(np.array(test_x))
  output = {0:'Cloudy',1:'Foggy',2:'Rainy',3:'Snowy',4:'Sunny'}
  plt.title(output[np.argmax(predict)])  
  plt.imshow(img)
  if output[np.argmax(predict)]!='Rainy':
    plt.savefig(dst+'/'+f'{name}')
    k+=1

**Trying Different Model**

Set All paths and define test generator

In [ ]:
dataset = '/content/Weather_detection_dataset/R_Train'
testset = '/content/Weather_detection_dataset/R_Test'
to_save = '/content/drive/MyDrive/weather_detection/trained_model'

"""# Dataset split"""

import keras
from keras.preprocessing.image import ImageDataGenerator

#Generate batches of tensor image data with real-time data augmentation, here used to rescale and split the dataset
#duplicates are removed by default
datagen = ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range=30,
    horizontal_flip=True
)

#Takes the path to a directory & generates batches of augmented data.
#When classes not provided, the list of classes is automatically inferred from the subdirectory names/structure
#default batch = 32
train_generator = datagen.flow_from_directory(
    directory = dataset,
    target_size = (224, 224)
)


test_datagen = ImageDataGenerator(
    rescale = 1. / 255,
)

test_generator = test_datagen.flow_from_directory(
    directory = testset,
    target_size = (224, 224)
)

classnames = [k for k,v in train_generator.class_indices.items()]
print("Image input %s" %str( train_generator.image_shape))
print("Classes: %r \n" %classnames)

print('Loaded %d training samples from %d classes.' %( train_generator.n, train_generator.num_classes))
print('Loaded %d test samples from %d classes.' %( test_generator.n, test_generator.num_classes))

Show some examples

In [ ]:
import matplotlib.pyplot as plt

x,y = train_generator.next()

for i in range(0,10):
    image = x[i]
    label = y[i].argmax() 
    print(classnames[label])
    plt.imshow(image)
    plt.show()

Create different models

In [ ]:
from tensorflow.python.keras import layers
from tensorflow.python.keras import Sequential

model = Sequential()

model.add(layers.Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu', input_shape=train_generator.image_shape))
model.add(layers.Conv2D(filters = 32, kernel_size = (3,3),padding = 'valid',  activation ='relu'))
model.add(layers.AveragePooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'valid', activation ='relu'))
model.add(layers.AveragePooling2D(pool_size=(2,2), strides=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = "relu", ))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(train_generator.num_classes, activation = "softmax"))
model.summary()


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

"""# Train the model"""

import time

#Total number of steps (batches of samples) to yield from generator before declaring one epoch finished
steps = train_generator.n//train_generator.batch_size

start = time.time()
try:
    history = model.fit_generator(train_generator, epochs=100, verbose=1,
                    steps_per_epoch = steps,
                    validation_data = test_generator,
                    validation_steps = test_generator.n//test_generator.batch_size,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=8, restore_best_weights=True)]
                    )
except KeyboardInterrupt:
    pass
end = time.time()
enc_time = end-start

print('Execution time:')
print(str(enc_time))
print()

"""# Save the model"""

filename = os.path.join(to_save, 'best1000.h5')
model.save(filename)
print("\nModel saved successfully on file %s\n" %filename)

Evaluate The Model

In [ ]:
"""# Evaluate the model

Accuracy on test set
"""

#needed to put shuffle = false
test_generator = test_datagen.flow_from_directory(
    directory = testset,
    target_size = (224, 224),
    shuffle=False
)

val_steps = test_generator.n//test_generator.batch_size
loss, acc = model.evaluate_generator( test_generator, verbose=1, steps = val_steps)
print('Test loss: %f' %loss)
print('Test accuracy: %f' %acc)

In [ ]:
"""Precision, recall, F-score"""

import sklearn.metrics 
from sklearn.metrics import classification_report, confusion_matrix

print('Loaded %d test samples from %d classes.' %(test_generator.n, test_generator.num_classes))

preds = model.predict_generator(test_generator, verbose=1, steps=val_steps)

Ypred = np.argmax(preds, axis=1)
Ytest = test_generator.classes  # shuffle=False in test_generator


In [ ]:
print(classification_report(Ytest[:-3], Ypred, labels=None, target_names=classnames, digits=3))

In [ ]:

"""Graphs"""

import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

"""confusion matrix"""

cm = confusion_matrix(Ytest[:-3], Ypred)
print(cm)

conf = [] # data structure for confusions: list of (i,j,cm[i][j])
for i in range(0,cm.shape[0]):
  for j in range(0,cm.shape[1]):
    if (i!=j and cm[i][j]>0):
      conf.append([i,j,cm[i][j]])

col=2
conf = np.array(conf)
conf = conf[np.argsort(-conf[:,col])]  # decreasing order by 3-rd column (i.e., cm[i][j])

print('\nConfusion matrix analysis' )
print('%-16s     %-16s  \t%s \t%s ' %('True','Predicted','errors','err %'))
print('------------------------------------------------------------------')
for k in conf:
  print('%-16s ->  %-16s  \t%d \t%.2f %% ' %(classnames[k[0]],classnames[k[1]],k[2],k[2]*100.0/test_generator.n))

**Try VGG16 with some changes**

In [ ]:
"""# Create the transfer-model"""

#feature extraction
pretrained_model =VGG16(input_shape=train_generator.image_shape,
                                               include_top=False,
                                               weights='imagenet')
pretrained_model.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()

**Trying Xception**

In [ ]:
from tensorflow.keras.applications import Xception
pretrained_model =Xception(input_shape=train_generator.image_shape,
                                               include_top=False,
                                               weights='imagenet')
pretrained_model.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()


**Trying with ResNet101**

In [ ]:
from tensorflow.keras.applications import ResNet101V2
pretrained_model =ResNet101V2(input_shape=train_generator.image_shape,
                                               include_top=False,
                                               weights='imagenet')
pretrained_model.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()

**ResNet50V2**

In [ ]:
from tensorflow.keras.applications import ResNet50V2
pretrained_model =ResNet50V2(input_shape=train_generator.image_shape,
                                               include_top=False,
                                               weights='imagenet')
pretrained_model.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()

**DenseNet201**

In [ ]:
from tensorflow.keras.applications import DenseNet201
pretrained_model =DenseNet201(input_shape=train_generator.image_shape,
                                               include_top=False,weights="imagenet")
pretrained_model.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()

**VGG19**

In [ ]:
from tensorflow.keras.applications import VGG19
pretrained_model = VGG19(input_shape=train_generator.image_shape,
                                               include_top=False,
                                weights="imagenet")
pretrained_model.trainable = False
model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()

Compiling and Fit the model

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

import time

#Total number of steps (batches of samples) to yield from generator before declaring one epoch finished
steps = train_generator.n//train_generator.batch_size

start = time.time()
try:
    history = model.fit_generator(train_generator, epochs=100, verbose=1,
                    steps_per_epoch = steps,
                    validation_data = test_generator,
                    validation_steps = test_generator.n//test_generator.batch_size,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=8, restore_best_weights=True)]
                    )
except KeyboardInterrupt:
    pass
end = time.time()
enc_time = end-start

print('Execution time:')
print(str(enc_time))
print()

"""# Save the model"""

filename = os.path.join(to_save, 'VGG19_transf2.h5')
model.save(filename)
print("\nModel saved successfully on file %s\n" %filename)

**Evalute the models**

In [ ]:
"""# Evaluate the model

Accuracy on test set
"""

#needed to put shuffle = false
test_generator = test_datagen.flow_from_directory(
    directory = testset,
    target_size = (224, 224),
    shuffle=False
)

val_steps = test_generator.n//test_generator.batch_size
loss, acc = model.evaluate_generator( test_generator, verbose=1, steps = val_steps)
print('Test loss: %f' %loss)
print('Test accuracy: %f' %acc)

In [ ]:
"""Precision, recall, F-score"""

import sklearn.metrics 
from sklearn.metrics import classification_report, confusion_matrix

print('Loaded %d test samples from %d classes.' %(test_generator.n, test_generator.num_classes))

preds = model.predict_generator(test_generator, verbose=1, steps=val_steps)

Ypred = np.argmax(preds, axis=1)
Ytest = test_generator.classes  # shuffle=False in test_generator

print(classification_report(Ytest[:-3], Ypred, labels=None, target_names=classnames, digits=3))

In [ ]:

"""Graphs"""

import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

"""confusion matrix"""

cm = confusion_matrix(Ytest[:-3], Ypred)
print(cm)

conf = [] # data structure for confusions: list of (i,j,cm[i][j])
for i in range(0,cm.shape[0]):
  for j in range(0,cm.shape[1]):
    if (i!=j and cm[i][j]>0):
      conf.append([i,j,cm[i][j]])

col=2
conf = np.array(conf)
conf = conf[np.argsort(-conf[:,col])]  # decreasing order by 3-rd column (i.e., cm[i][j])

print('\nConfusion matrix analysis' )
print('%-16s     %-16s  \t%s \t%s ' %('True','Predicted','errors','err %'))
print('------------------------------------------------------------------')
for k in conf:
  print('%-16s ->  %-16s  \t%d \t%.2f %% ' %(classnames[k[0]],classnames[k[1]],k[2],k[2]*100.0/test_generator.n))


**Blind Test**

In [ ]:
# """# blind test"""

# files = '/content/drive/My Drive/WeatherBlindTestSet'

# test_generator = test_datagen.flow_from_directory(
#     directory = files,
#     target_size = (224, 224),
#     shuffle=False
# )


# val_steps = test_generator.n//test_generator.batch_size+1
# preds = model.predict_generator(test_generator, verbose=1, steps=val_steps)

# print(preds)

# Ypred = np.argmax(preds, axis=1)
# np.set_printoptions(threshold=np.inf)


# res = []
# sun = 0
# haz = 0 
# ran = 0 
# sn = 0;
# for i in Ypred:
#   if i == 0:
#     res.append('HAZE')
#     haz += 1
#   if i == 1:
#     res.append('RAINY')
#     ran += 1
#   if i == 2:
#     res.append('SNOWY')
#     sn += 1
#   if i == 3:
#     res.append('SUNNY')
#     sun += 1

# print(res)
# print(Ypred)

# print(len(res))
# print(len(Ypred))
# print(sun)
# print(haz)
# print(ran)
# print(sn)




# results = pd.DataFrame(res, columns=['predictions'])
# from google.colab import files
# results.to_csv('prediction.csv') 
# files.download('prediction.csv')